In [1]:
from fastai.vision.all import *
from random import randint
from ml_utils import *
import os

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
clips_df = pd.read_csv('data/labels.csv')

# dropping dangling columns
clips_df = clips_df.loc[:, ~clips_df.columns.str.contains('^Unnamed')]

def clean_labels(c): return str([(i).strip().replace(" ", "_").replace("-", "_").replace("/", "_") for i in re.sub(r"[\[\]]",'',c).split(',')]).replace("'","").strip("'").strip("[").strip("]").replace(" ","")
clips_df['tools_present'] = clips_df['tools_present'].apply(clean_labels)
clips_df.head()

,clip_name,tools_present
0,clip_000000,"needle_driver,nan,needle_driver,cadiere_forceps"
1,clip_000001,"needle_driver,nan,needle_driver,cadiere_forceps"
2,clip_000002,"needle_driver,nan,needle_driver,cadiere_forceps"
3,clip_000003,"needle_driver,nan,needle_driver,cadiere_forceps"
4,clip_000004,"needle_driver,nan,needle_driver,cadiere_forceps"


In [4]:
df=clips_df.groupby('tools_present').nth(0)
len(df)

153

In [ ]:
src=Path('data/train_images_crop')
dst=Path('data/testing/images')

In [ ]:
for i in get_image_files(dst): os.remove(i) 

In [ ]:
for clip in df.clip_name:
    fs = get_image_files(src/clip)
    if len(fs)>0:
        fno=randint(0,len(fs)-1)
        im=PILImage.create(fs[fno])
        fn=str(dst)+'/'+clip+'_'+fs[fno].name
        im.save(fn)
    

In [ ]:
ensem_path=Path("test/algorithm/cls")

In [ ]:
ensem_learner=[load_learner(m, cpu=True) for m in ensem_path.ls() if m.suffix=='.pkl']

In [ ]:
fs=get_image_files(dst);len(fs)

In [ ]:
tta_res=[]
prs_items=[]
for learn in ensem_learner:
    tta_res.append(learn.tta(dl=learn.dls.test_dl(fs)))
    if len(prs_items)<1:
        prs_items=learn.dl.items

tta_prs=first(zip(*tta_res))
tta_prs=torch.stack(tta_prs)

lbls=[]
for i in range(len(c)):
    arm_preds = tta_prs[:,:,cfg(i):cfg(i+1)].mean(0);
    arm_idxs = arm_preds.argmax(dim=1)
    arm_vocab = np.array(vocab[i])
    lbls.append(arm_vocab[arm_idxs])


In [ ]:
data=[]
for lbl,f in zip(lbls,prs_items):
    l1,l2,l3,l4=lbls
    data.append((f,l1+','+l2+','+l3+','+l4))
df = pd.DataFrame(data,columns=['image_id','PR_labels'])
df

In [ ]:
data=[]
for l1,l2,l3,l4,f in zip(lbls[0],lbls[1],lbls[2],lbls[3],prs_items):
    print((f,l1+','+l2+','+l3+','+l4))
    data.append((f,l1+','+l2+','+l3+','+l4))
    data.append((f,l1+','+l2+','+l3+','+l4))

In [ ]:
clips_df.set_index('clip_name',inplace=True,drop=True)

In [ ]:
def get_GT_labels(f):
    c=str(Path(f).name).rpartition('_')[0]
    return clips_df.loc[c,'tools_present']

In [ ]:
df = pd.DataFrame(data,columns=['image_id','PR_labels'])
# df['GT_labels']=df['image_id'].map(get_GT_labels) 
# df['accurate_tf'] = (df['PR_labels']==df['GT_labels'])
df

In [ ]:
df = pd.DataFrame(data,columns=['image_id','PR_labels'])
df['GT_labels']=df['image_id'].map(get_GT_labels) 
df['accurate_tf'] = (df['PR_labels']==df['GT_labels'])
df